# Cài đặt các thư viện cần thiết để cào

In [1]:
%pip install requests-html beautifulsoup4 pandas numpy lxml lxml_html_clean nest_asyncio pyppeteer-py3 pyppeteer requests selenium cloudscraper selenium webdriver-manager

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import datetime
import pandas as pd 
import numpy as np
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup

# Cào link

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Đường dẫn tới ChromeDriver
# chromedriver_path = 'C:\\chromedriver\\chromedriver.exe'

# # Cấu hình ChromeDriver service
# service = Service(chromedriver_path)

# # Khởi tạo WebDriver cho Cốc Cốc
#options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # Chạy ẩn danh (không hiển thị trình duyệt)

try:
    
    # Danh sách để lưu trữ các link
    arr = []

    # Lặp qua các trang
    for page in range(1, 3):
        driver = webdriver.Chrome()
        url = f"https://www.nhatot.com/mua-ban-can-ho-chung-cu?page={page}"
        print(f"Đang lấy dữ liệu từ trang: {url}")
        # Mở trang web
        driver.get(url)
        # Lấy tất cả các đường link sản phẩm
        links = driver.find_elements(By.CSS_SELECTOR,"a.AdItem_adItem__gDDQT")
        for link in links:
            arr.append(link.get_attribute('href'))
        print(f"Số lượng link đã lấy: {len(arr)}")
        print("-" * 50)
        # Đóng trình duyệt
        driver.quit()
except Exception as e:
    print(f"Có lỗi xảy ra: {str(e)}")
# Tạo DataFrame từ danh sách các link
df = pd.DataFrame(arr, columns=['links'])
# Lưu DataFrame vào file CSV
df.to_csv('../data/test/links_chotot.csv', index=False)
print("Hoàn thành lấy dữ liệu!")


Đang lấy dữ liệu từ trang: https://www.nhatot.com/mua-ban-can-ho-chung-cu?page=1
Số lượng link đã lấy: 25
--------------------------------------------------
Đang lấy dữ liệu từ trang: https://www.nhatot.com/mua-ban-can-ho-chung-cu?page=2
Số lượng link đã lấy: 50
--------------------------------------------------
Hoàn thành lấy dữ liệu!


In [7]:
frame = pd.read_csv('../data/test/links_chotot.csv')
frame.head(20)
frame.info

<bound method DataFrame.info of                                                 links
0   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
1   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
2   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
3   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
4   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
5   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
6   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
7   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
8   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
9   https://www.nhatot.com/mua-ban-can-ho-chung-cu...
10  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
11  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
12  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
13  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
14  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
15  https://www.nhatot.com/mua-ban-can-ho-chung-cu...
16  https://www.nhatot.com/mua-ban-can-ho-chung-cu

# Cào dữ liệu

In [8]:
address=[]
estate_condition,price_m2, main_direction,floor,id_house=[],[],[],[],[]
balcony_direction,Legal_papers,Character,acreage,bedroom,block=[],[],[],[],[],[]
bath_room,types,furniture_condition=[],[],[]
person,price=[],[]
items =['Tình trạng bất động sản:','Diện tích:','Giá/m2:','Số phòng ngủ:','Tên phân khu/Lô/Block/Tháp:','Tầng số:',
       'Số phòng vệ sinh:','Loại hình căn hộ:','Giấy tờ pháp lý:','Mã căn','Tình trạng nội thất:','Hướng cửa chính:',
       'Hướng ban công','Đặc điểm căn hộ']

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from requests_html import HTML

def getdata(link):
    # Cấu hình Selenium WebDriver
    options = Options()
    # options.add_argument("--headless")  # Chạy trình duyệt ở chế độ headless (không hiển thị giao diện)
    options.add_argument("--disable-gpu")  # Tắt GPU (không cần thiết cho chế độ headless)
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    # Tạo instance của trình điều khiển Chrome
    driver = webdriver.Chrome(options=options)
        # Mở trang web        
    driver.get(link)
        
        # Lấy nội dung HTML của trang
    html_text = driver.page_source
    driver.quit()  # Đóng trình duyệt sau khi lấy dữ liệu
        
        # Phân tích cú pháp HTML để trích xuất dữ liệu
    r_html = HTML(html=html_text)
    if len(r_html.find(".notfound-message")) == 0:
        arr = []
        # col-md-8 là chứa toàn bộ web
        r_table = r_html.find(".col-md-8")
        if len(r_table) > 0:
            # test là toàn bộ mã html của trang đang truy xuất
            test = r_table[0]
            inf = test.find('.col-xs-12')
            price = test.find('.priceWrapper___y7Aj6')
            temp = r_html.find('a')
            col = temp[6].find('b')
            info = inf[1].text + '\n'
            for item in r_table:
                print(item)
            print(test.html)  # In mã HTML thô của test
            print("\nLength of test element (text):")
            print(len(test.text))
            for item in inf:
                print(item)
            for item in price:
                print(item)
            for item in temp:
                print(item)
            for item in col:
                print(item)
            for item in info:
                print(item)
            arr.append(info)  # dia chi nha
            arr.append(inf[3].text)  # thong tin can nha
            arr.append(price[1].text)  # gia nha
        else:
            return []

        str1 = arr
        str1 = [arr]
        temp = arr[1].split('\n')
        idx = []
        idx.append([arr[0].split('\n')[0]])
        for i in temp:
            idx.append(i.split('\n'))
            idx.append([arr[2]])
        return idx
    else:
        return []




In [10]:
for link in frame['links']:
    print(link)
    data = getdata(link)
    dem =0 
    if (len(data)>0):
        address+=data[0]
        for item in items:
            mang=[]
            flag=0
            for i in data:
                if item in i[0]:
                    mang.append(i[0].split(':')[1])
                    flag = 1
            dem = dem+1
            if flag==1:
                if (dem ==1): estate_condition+=mang
                elif (dem ==2): acreage+=mang
                elif (dem ==3): price_m2+=mang
                elif (dem ==4): bedroom+=mang
                elif (dem ==5): block+=mang
                elif (dem ==6): floor+=mang
                elif (dem ==7): bath_room+=mang
                elif (dem ==8): types+=mang
                elif (dem ==9): Legal_papers+=mang
                elif (dem ==10): id_house+=mang
                elif (dem ==11): furniture_condition+=mang
                elif (dem ==12): main_direction+=mang
                elif (dem ==13): balcony_direction+=mang
                elif (dem ==14): Character+=mang
            else:
                if (dem ==1): estate_condition.append(np.nan)
                elif (dem ==2): acreage.append(np.nan)
                elif (dem ==3): price_m2.append(np.nan)
                elif (dem ==4): bedroom.append(np.nan)
                elif (dem ==5): block.append(np.nan)
                elif (dem ==6): floor.append(np.nan)
                elif (dem ==7): bath_room.append(np.nan)
                elif (dem ==8): types.append(np.nan)
                elif (dem ==9): Legal_papers.append(np.nan)
                elif (dem ==10): id_house.append(np.nan)
                elif (dem ==11): furniture_condition.append(np.nan)
                elif (dem ==12): main_direction.append(np.nan)
                elif (dem ==13): balcony_direction.append(np.nan)
                elif (dem ==14): Character.append(np.nan)  
        price+=data[len(data)-1]

https://www.nhatot.com/mua-ban-can-ho-chung-cu-thanh-pho-di-an-binh-duong/117282681.htm#px=SR-stickyad-[PO-1][PL-top]
<Element 'div' class=('col-md-8',)>
<div class="col-md-8"><div class="DetailView_adviewCointainer__rdzwn"><div><div class="AdImage_adImageWrapper__HkN6D" style="background-image:url(https://static.chotot.com/storage/logo/svg/chotot-logo-text.svg);background-position:center center;background-repeat:no-repeat;background-color:#eee;background-size:50% auto"><div id="adview-carousel-placeholder" style="display: none; position: absolute; overflow: scroll; width: 100%; max-height: 450px;"><img src="https://cdn.chotot.com/5oy3dxg3cq114H9T3NeHmTcCTpR4JevHYjOoSdmVEjE/preset:view/plain/78433e072fbbfefe38ee1b2b4ef0dab0-2883927222624293276.jpg" alt="https://cdn.chotot.com/5oy3dxg3cq114H9T3NeHmTcCTpR4JevHYjOoSdmVEjE/preset:view/plain/78433e072fbbfefe38ee1b2b4ef0dab0-2883927222624293276.jpg" width="100%" height="auto" style="object-fit:contain"/><img src="https://cdn.chotot.com/O0pp6

IndexError: list index out of range

In [11]:
df = {
        'DiaChi' : address,
        'TinhTrangBDS' : estate_condition,
        'DienTich' : acreage,
        'Gia/m2'   : price_m2,
        'Phongngu': bedroom,
      'TenPhanKhu': block,
      'SoTang': floor,
      'PhongTam':bath_room,
      'Loai': types,
      'GiayTo': Legal_papers,
      'MaCanHo': id_house,
      'TinhTrangNoiThat': furniture_condition,
      'HuongCuaChinh': main_direction,
      'HuongBanCong': balcony_direction,
      'DacDiem':Character,
      'Gia': price,
}
df = pd.DataFrame(df)
df.to_csv('../data/test/raw_data.csv',encoding="utf-8-sig",index=False)
df

,DiaChi,TinhTrangBDS,DienTich,Gia/m2,Phongngu,TenPhanKhu,SoTang,PhongTam,Loai,GiayTo,MaCanHo,TinhTrangNoiThat,HuongCuaChinh,HuongBanCong,DacDiem,Gia
